<a href="https://colab.research.google.com/github/KanoonGammy/Python-for-investing/blob/All/5_Workshop_Create_Backtesting_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install mplfinance

In [ ]:
import yfinance as yf
import pandas as pd
import mplfinance as mpf
import matplotlib.pyplot as mpl
import datetime

In [ ]:
df = yf.download('BTC-USD')

[*********************100%***********************]  1 of 1 completed


In [ ]:
def macd_strategy(df , fast=12 , slow=26 , signal = 9):
  #create MACD
  df['fast'] = df.Close.ewm(span = fast , adjust = False , min_periods = fast +1 ).mean()
  df['slow'] = df.Close.ewm(span = slow , adjust = False , min_periods = slow +1 ).mean()
  df['MACD'] = df.fast - df.slow
  df['signal'] = df.MACD.rolling(signal).mean()
  df['histogram'] = df.MACD - df.signal

  #create entry point
  df.loc[(df['histogram'] > 0) & (df['histogram'].shift(1)< 0),'action'] = 'buy'
  df.loc[(df['histogram'] < 0) & (df['histogram'].shift(1)> 0),'action'] = 'sell'  

  #create marker_position
  df.loc[df['action'] == 'buy' , 'marker_position'] = df.Low * 0.95
  df.loc[df['action'] == 'sell' , 'marker_position'] = df.High * 1.05
  
  return df

In [ ]:
def strategy_return(df , action_price = 'Open' , long_or_short = 'long'):
  action_df = df.loc[df['action'].notnull()]

  #Condition of action_price
  if(action_price == 'Open'):
    action_df['action_price'] = df['Open'].shift(-1)
    action_df['returns'] = action_df['action_price'].pct_change()

  elif(action_price == 'Close'):
    action_df['action_price'] = df['Close']
    action_df['returns'] = action_df['action_price'].pct_change()

  elif(action_price == 'Adj Close'):
    action_df['action_price'] = df['Adj Close']
    action_df['returns'] = action_df['action_price'].pct_change()
  else:
    raise ValueError('Not Correct action_price , choose between {Open , Close , Adj Close}')

  #modify data to find return that on the same "sell" row 
  record_df = action_df[['action' , 'action_price' , 'returns']] #Choose columns
  record_df.reset_index(inplace=True) #reset index , to use
  record_df['previous_signal_date'] = record_df['Date'].shift(1) #เรา reset_index เพื่อสิ่งนี้!!
  record_df['previous_action'] = record_df['action'].shift(1) # จัดเพื่อเช็ต
  record_df['previous_action_price'] = record_df['action_price'].shift(1) #จัดเพื่อดูราคาเข้าซื้อ
  record_df.rename(columns={'Date':'signal_date'}, inplace=True) # rename

  #ทำ long , short 
  if long_or_short == 'long' :
    summary_df = record_df[record_df.action == 'sell']
    print('long strategy')
  elif long_or_short == 'short' :
    summary_df = record_df[summary_df.action == 'buy']
    summary_df['returns'] = summary_df['returns'] * -1 #ต้องคูณนะ เพราะค่าติดลบ เราหากำไรขา shoty
    print('short strategy')
  else:
    raise ValueError('Not Correct long_or_short , choose between {long , short}')
  return action_df , record_df , summary_df

In [ ]:
df = macd_strategy(df)

In [ ]:
df = strategy_return(df , action_price = 'Open' , long_or_short = 'long')

In [ ]:
df

(                                   Open          High           Low  \
 Date                                                                  
 2014-10-24 00:00:00+00:00    358.591003    364.345001    353.304993   
 2014-11-06 00:00:00+00:00    339.458008    352.966003    338.424011   
 2014-11-20 00:00:00+00:00    380.307007    382.024994    356.781006   
 2014-11-29 00:00:00+00:00    376.152008    387.601013    372.144989   
 2014-12-06 00:00:00+00:00    376.756989    378.447998    370.945007   
 ...                                 ...           ...           ...   
 2022-11-23 00:00:00+00:00  16195.588867  16638.193359  16170.502930   
 2022-12-18 00:00:00+00:00  16795.609375  16815.386719  16697.820312   
 2022-12-26 00:00:00+00:00  16842.250000  16920.123047  16812.369141   
 2022-12-28 00:00:00+00:00  16716.400391  16768.169922  16497.556641   
 2023-01-03 00:00:00+00:00  16688.847656  16760.447266  16622.371094   
 
                                   Close     Adj Close       V

In [ ]:
df[0]

,Open,High,Low,Close,Adj Close,Volume,fast,slow,MACD,signal,histogram,action,marker_position,action_price,returns
Date,,,,,,,,,,,,,,,
2014-10-24 00:00:00+00:00,358.591003,364.345001,353.304993,358.345001,358.345001,15585700,375.767142,381.157050,-5.389908,-4.274699,-1.115209,sell,382.562251,358.610992,NaN
2014-11-06 00:00:00+00:00,339.458008,352.966003,338.424011,349.290009,349.290009,18797000,342.638116,354.717992,-12.079876,-12.828999,0.749123,buy,321.502811,349.817993,-0.024520
2014-11-20 00:00:00+00:00,380.307007,382.024994,356.781006,357.839996,357.839996,25233200,375.714420,371.014360,4.700060,6.680873,-1.980812,sell,401.126244,357.878998,0.023043
2014-11-29 00:00:00+00:00,376.152008,387.601013,372.144989,375.490997,375.490997,15375600,371.921286,370.260658,1.660627,1.271219,0.389409,buy,353.537740,375.510010,0.049265
2014-12-06 00:00:00+00:00,376.756989,378.447998,370.945007,374.785004,374.785004,7009320,374.612070,372.703850,1.908220,2.092358,-0.184138,sell,397.370398,374.835999,-0.001795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-23 00:00:00+00:00,16195.588867,16638.193359,16170.502930,16610.707031,16610.707031,32958875628,16777.504372,17656.549068,-879.044696,-903.486544,24.441848,buy,15361.977783,16611.636719,-0.104193
2022-12-18 00:00:00+00:00,16795.609375,16815.386719,16697.820312,16757.976562,16757.976562,10924354698,17062.710900,17133.553017,-70.842117,-47.650637,-23.191480,sell,17656.156055,16759.041016,0.008874
2022-12-26 00:00:00+00:00,16842.250000,16920.123047,16812.369141,16919.804688,16919.804688,11886957804,16890.259102,16986.740758,-96.481656,-106.390051,9.908396,buy,15971.750684,16919.291016,0.009562


In [ ]:
df[1]

,signal_date,action,action_price,returns,previous_signal_date,previous_action,previous_action_price
0,2014-10-24 00:00:00+00:00,sell,358.610992,NaN,NaT,NaN,NaN
1,2014-11-06 00:00:00+00:00,buy,349.817993,-0.024520,2014-10-24 00:00:00+00:00,sell,358.610992
2,2014-11-20 00:00:00+00:00,sell,357.878998,0.023043,2014-11-06 00:00:00+00:00,buy,349.817993
3,2014-11-29 00:00:00+00:00,buy,375.510010,0.049265,2014-11-20 00:00:00+00:00,sell,357.878998
4,2014-12-06 00:00:00+00:00,sell,374.835999,-0.001795,2014-11-29 00:00:00+00:00,buy,375.510010
...,...,...,...,...,...,...,...
217,2022-11-23 00:00:00+00:00,buy,16611.636719,-0.104193,2022-11-08 00:00:00+00:00,sell,18543.761719
218,2022-12-18 00:00:00+00:00,sell,16759.041016,0.008874,2022-11-23 00:00:00+00:00,buy,16611.636719
219,2022-12-26 00:00:00+00:00,buy,16919.291016,0.009562,2022-12-18 00:00:00+00:00,sell,16759.041016
220,2022-12-28 00:00:00+00:00,sell,16552.322266,-0.021689,2022-12-26 00:00:00+00:00,buy,16919.291016


In [ ]:
df[2]

,signal_date,action,action_price,returns,previous_signal_date,previous_action,previous_action_price
0,2014-10-24 00:00:00+00:00,sell,358.610992,NaN,NaT,NaN,NaN
2,2014-11-20 00:00:00+00:00,sell,357.878998,0.023043,2014-11-06 00:00:00+00:00,buy,349.817993
4,2014-12-06 00:00:00+00:00,sell,374.835999,-0.001795,2014-11-29 00:00:00+00:00,buy,375.510010
6,2014-12-30 00:00:00+00:00,sell,310.914001,-0.035285,2014-12-24 00:00:00+00:00,buy,322.286011
8,2015-01-03 00:00:00+00:00,sell,281.145996,-0.122611,2014-12-31 00:00:00+00:00,buy,320.434998
...,...,...,...,...,...,...,...
212,2022-09-18 00:00:00+00:00,sell,19418.572266,-0.091610,2022-09-09 00:00:00+00:00,buy,21376.912109
214,2022-10-11 00:00:00+00:00,sell,19052.646484,-0.019309,2022-09-28 00:00:00+00:00,buy,19427.779297
216,2022-11-08 00:00:00+00:00,sell,18543.761719,-0.051493,2022-10-17 00:00:00+00:00,buy,19550.466797
218,2022-12-18 00:00:00+00:00,sell,16759.041016,0.008874,2022-11-23 00:00:00+00:00,buy,16611.636719


In [ ]:
df[2]['returns'].mean()

0.054963937163658126

In [ ]:
df[2]['returns'].max()

1.1621341336774447

In [ ]:
df[2]['returns'].min()

-0.14383670488627054